[See docs/DISCLAIMER_SNIPPET.md](../../../docs/DISCLAIMER_SNIPPET.md)

This notebook is licensed under the Apache License 2.0 (SPDX-License-Identifier: Apache-2.0).
# Alpha-Factory v1 · World Model Demo
This Colab notebook boots the `alpha_asi_world_model` demo in minutes. Follow each step below.

## 1. Clone the repository

In [ ]:
!git clone https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git && cd AGI-Alpha-Agent-v0 && git checkout a93de700289e72950a758ffa10ca05a54ddd6a17
%cd AGI-Alpha-Agent-v0

## 2. Install dependencies
Run the cell below to install Python packages. This may take a few minutes.

In [ ]:
!pip install -r alpha_factory_v1/demos/alpha_asi_world_model/requirements.txt

## 3. Launch the demo
Start the FastAPI server with all agents active. The service exposes REST and WebSocket endpoints on port 7860.

In [ ]:
!python -m alpha_factory_v1.demos.alpha_asi_world_model.alpha_asi_world_model_demo --demo --host 0.0.0.0 --port 7860 &

## 4. Open the dashboard
Use the utility below to open the interface in a new tab.

In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(7860)

## 5. Interact via REST
You can control the orchestrator programmatically. The example below lists the registered agents then spawns a new environment.

In [ ]:
import requests, json
base='http://localhost:7860'
print('Active agents:', requests.get(base+'/agents').json())
resp = requests.post(base+'/command', json={'cmd':'new_env'})
print('Spawned:', resp.json())

## 6. Shut down
Stop the service when you are done.

In [ ]:
!fuser -k 7860/tcp

## Optional offline mode
Set `NO_LLM=1` if you do not provide an API key.

In [ ]:
import os
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['NO_LLM']='1'

In [ ]:
from IPython.display import IFrame
IFrame('http://localhost:7860', width='100%', height=480)

## 7. Query via OpenAI Agents SDK

In [ ]:
from openai_agents import Agent, AgentRuntime, Tool
import requests

@Tool(name='list_agents', description='List orchestrator agents')
async def list_agents():
    return requests.get('http://localhost:7860/agents').json()

class Inspector(Agent):
    name='inspector'
    tools=[list_agents]
    async def policy(self, obs, ctx):
        return await self.tools.list_agents()

rt=AgentRuntime(api_key=None)
rt.register(Inspector())
print('Active:', await list_agents())

## 8. Optional ADK gateway
Set `ALPHA_FACTORY_ENABLE_ADK=true` to expose the inspector agent via Google ADK.

In [ ]:
!ALPHA_FACTORY_ENABLE_ADK=true python openai_agents_bridge.py &